<a href="https://colab.research.google.com/github/Shohei-Kinoshita/prob-space-accommodation-charge-prediction/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# カレントディレクトリの指定
%cd /content/drive/MyDrive/分析コンペ/05_ProbSpace/accommodation_charge_prediction

Mounted at /content/drive
/content/drive/MyDrive/分析コンペ/05_ProbSpace/accommodation_charge_prediction


In [ ]:
! nvidia-smi

Mon Mar 14 11:20:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import random
import os
import datetime
import numpy as np
import pandas as pd
import warnings
import gc
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from src.config import *
import src.preprocessing as pr

warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('input/train_data.csv', parse_dates=[COL_LAST_REVIEW], dtype=DICT_DTYPES)
df_test = pd.read_csv('input/test_data.csv', parse_dates=[COL_LAST_REVIEW], dtype=DICT_DTYPES)
df_train_station_info = pd.read_csv('input/train_data_distance_from_station.csv', dtype=DICT_DTYPES)
df_test_station_info = pd.read_csv('input/test_data_distance_from_station.csv', dtype=DICT_DTYPES)
df_train_svd_name = pd.read_csv('input/train_data_name_features.csv')
df_test_svd_name = pd.read_csv('input/test_data_name_features.csv')
#df_train_name_emb = pd.read_csv('input/train_data_name_embedding.csv')
#df_test_name_emb = pd.read_csv('input/test_data_name_embedding.csv')
df_station_list = pd.read_csv('input/station_list.csv')
sample_sub = pd.read_csv('input/submission.csv')

In [ ]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 70
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 11
EARLY_STOP = True

LIST_USE_COL = [COL_NEIGHBOURHOOD,
                COL_LATITUDE,
                COL_LONGITUDE,
                COL_ROOM_TYPE,
                COL_MINIMUM_NIGHTS,
                COL_NUMBER_OF_REVIEWS,
                'elapsed_days',
                COL_REVIEWS_PER_MONTH,
                COL_AVAILABILITY_365]

LIST_LABEL_ENC = [COL_NEIGHBOURHOOD,
                  COL_ROOM_TYPE]

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, features, target):
        super().__init__()
        self.features = features.reset_index(drop=True)
        self.target = target.reset_index(drop=True)

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        x = torch.tensor(self.features.loc[idx, :].values, dtype=torch.float)
        y = torch.tensor(self.target[idx], dtype=torch.float)
        return x, y


class TestDataset(Dataset):
    def __init__(self, features):
        super().__init__()
        self.features = features.reset_index(drop=True)
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        x = torch.tensor(self.features.loc[idx, :].values, dtype=torch.float)
        return x


# class Model(nn.Module):
#     def __init__(self, num_features, num_targets, hidden_size_1, hidden_size_2):
#         super(Model, self).__init__()
#         self.batch_norm1 = nn.BatchNorm1d(num_features)
#         self.dropout1 = nn.Dropout(0.2)
#         self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size_1))
        
#         self.batch_norm2 = nn.BatchNorm1d(hidden_size_1)
#         self.dropout2 = nn.Dropout(0.2)
#         self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size_1, hidden_size_2))
        
#         self.batch_norm3 = nn.BatchNorm1d(hidden_size_2)
#         self.dropout3 = nn.Dropout(0.25)
#         self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size_2, num_targets))
    
#     def forward(self, x):
#         x = self.batch_norm1(x)
#         x = self.dropout1(x)
#         x = F.relu(self.dense1(x))
        
#         x = self.batch_norm2(x)
#         x = self.dropout2(x)
#         x = F.relu(self.dense2(x))
        
#         x = self.batch_norm3(x)
#         x = self.dropout3(x)
#         x = self.dense3(x)
        
#         return x


class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size_1, hidden_size_2):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size_1))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size_1)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size_1, hidden_size_2))

        self.batch_norm3 = nn.BatchNorm1d(hidden_size_2)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size_2, hidden_size_2))
        
        self.batch_norm4 = nn.BatchNorm1d(hidden_size_2)
        self.dropout4 = nn.Dropout(0.25)
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size_2, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.relu(self.dense3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x

In [ ]:
def train_fn(train_loader, model, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    train_rmsle = 0.0

    for features, labels in tqdm(train_loader):
        optimizer.zero_grad()  # 勾配情報を初期化
        features = features.to(device)  # GPU上に転送
        labels = labels.to(device)  # GPU上に転送
        outputs = model(features)  # GPU上に転送
        outputs = torch.squeeze(outputs)  # 1次元に変換
        loss = torch.sqrt(criterion(outputs, labels))  # 損失を計算
        preds = outputs.detach().cpu().numpy()  # numpyの配列に変換
        labels = labels.detach().cpu().numpy()  # numpyの配列に変換

        train_loss += loss.item()  # 値だけ取得
        train_rmsle += np.sqrt(mean_squared_error(labels, preds))  # 評価指標の計算
        loss.backward()  # 勾配を計算
        optimizer.step()  # 勾配を更新

    train_loss /= len(train_loader)  # 1epoch分の平均を計算
    train_rmsle /= len(train_loader)  # 1epoch分の平均を計算

    return train_loss, train_rmsle


def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    valid_loss = 0.0
    valid_rmsle = 0.0
    valid_preds = []

    for features, labels in tqdm(valid_loader):
        features = features.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            outputs = model(features)
            outputs = torch.squeeze(outputs)
            loss = torch.sqrt(criterion(outputs, labels))
            preds = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

        valid_loss += loss.item()
        valid_rmsle += np.sqrt(mean_squared_error(labels, preds))
        valid_preds.append(preds)

    valid_loss /= len(valid_loader)
    valid_rmsle /= len(valid_loader)
    valid_preds = np.concatenate(valid_preds)

    return valid_loss, valid_rmsle, valid_preds


def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for features in dataloader:
        features = features.to(device)

        with torch.no_grad():
            outputs = model(features)
        
        preds.append(outputs.detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=42)

In [ ]:
df_all = pd.concat([
    df_train,
    df_test
]).reset_index(drop=True)

In [ ]:
# 2020.4.30からの経過日数を特徴量として追加
df_all[COL_ELAPSED_DAYS] = (datetime.datetime(2020, 4, 30) - df_all[COL_LAST_REVIEW]).dt.days

In [ ]:
df_all.fillna(0, inplace=True)

In [ ]:
# for col in LIST_LABEL_ENC:
#     le = LabelEncoder()
#     df_all[col] = le.fit_transform(df_all[col])

In [ ]:
df_all = df_all[LIST_USE_COL]

In [ ]:
df_all = pd.get_dummies(df_all, columns=LIST_LABEL_ENC, drop_first=True)

In [ ]:
df_all.head()

,latitude,longitude,minimum_nights,number_of_reviews,elapsed_days,reviews_per_month,availability_365,neighbourhood_Arakawa Ku,neighbourhood_Bunkyo Ku,neighbourhood_Chiyoda Ku,...,neighbourhood_Shibuya Ku,neighbourhood_Shinagawa Ku,neighbourhood_Shinjuku Ku,neighbourhood_Suginami Ku,neighbourhood_Sumida Ku,neighbourhood_Taito Ku,neighbourhood_Toshima Ku,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,35.68185,139.80310,1,55,5.0,2.21,173,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35.72063,139.78536,6,72,36.0,2.11,9,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,35.74723,139.82349,1,18,38.0,3.46,288,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,35.68456,139.68077,1,2,28.0,1.76,87,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,35.69840,139.70467,1,86,91.0,2.00,156,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
df_all_station_info = pd.concat([df_train_station_info, df_test_station_info], axis=0).reset_index(drop=True)
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA(n_components=10, random_state=0))
])

features_tmp = pipe.fit_transform(df_all_station_info)
df_distance_features = pd.DataFrame(features_tmp, columns=[f'PCA_{i+1}' for i in range(features_tmp.shape[1])])

In [ ]:
df_train_distance_features = df_distance_features[:df_train.shape[0]].reset_index(drop=True)
df_test_distance_features = df_distance_features[df_train.shape[0]:].reset_index(drop=True)

In [ ]:
X = df_all[:df_train.shape[0]].reset_index(drop=True)
X = pd.concat([X, df_train_distance_features, df_train_svd_name], axis=1)
y = np.log1p(df_train[COL_Y])

X_inference = df_all[df_train.shape[0]:].reset_index(drop=True)
X_inference = pd.concat([X_inference, df_test_distance_features, df_test_svd_name], axis=1)

In [ ]:
scale = StandardScaler()
scale.fit(X)
X = pd.DataFrame(data=scale.transform(X), columns=X.columns)
X_inference = pd.DataFrame(data=scale.transform(X_inference), columns=X_inference.columns)

In [ ]:
X.head()

,latitude,longitude,minimum_nights,number_of_reviews,elapsed_days,reviews_per_month,availability_365,neighbourhood_Arakawa Ku,neighbourhood_Bunkyo Ku,neighbourhood_Chiyoda Ku,...,name_nmf_41,name_nmf_42,name_nmf_43,name_nmf_44,name_nmf_45,name_nmf_46,name_nmf_47,name_nmf_48,name_nmf_49,name_nmf_50
0,-0.452253,1.100902,-0.274144,0.715964,-0.752875,0.559342,0.136454,-0.171366,-0.110264,-0.13584,...,-0.17204,-0.163183,-0.447489,-0.146665,-0.210688,-0.189127,-0.202038,-0.206581,-0.149963,-0.193944
1,0.514113,0.777728,0.313812,1.134846,-0.458280,0.489338,-1.280089,-0.171366,-0.110264,-0.13584,...,-0.17204,-0.163183,-0.447489,-0.146665,-0.210688,-0.189127,-0.202038,-0.206581,-0.149963,-0.224481
2,1.176963,1.472351,-0.274144,-0.195720,-0.439274,1.434393,1.129761,-0.171366,-0.110264,-0.13584,...,-0.17204,-0.163183,2.433680,-0.146665,-0.210688,0.095166,-0.202038,-0.206581,-0.149963,-0.309818
3,-0.384722,-1.127610,-0.274144,-0.589961,-0.534304,0.244324,-0.606367,-0.171366,-0.110264,-0.13584,...,-0.17204,0.140256,2.310672,-0.146665,-0.210688,9.413826,-0.202038,-0.206581,-0.149963,-0.309818
4,-0.039840,-0.692218,-0.274144,1.479807,0.064390,0.412334,-0.010383,-0.171366,-0.110264,-0.13584,...,-0.17204,0.006835,-0.447489,-0.146665,7.724682,-0.189127,-0.202038,-0.094801,-0.149963,-0.309818


In [ ]:
num_features= X.shape[1]
num_targets= 1
hidden_size_1=512
hidden_size_2=256

In [ ]:
kf = KFold(n_splits=NFOLDS)
preds_oof = np.zeros(X.shape[0])
for i, (train_idx, valid_idx) in enumerate(kf.split(X)):
    model = Model(num_features, num_targets, hidden_size_1, hidden_size_2)
    model.to(DEVICE)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(params=model.parameters(), lr=0.001)
    print(f'fold: {i}')
    print('='*50)
    X_train, X_valid = X.loc[train_idx, :], X.loc[valid_idx, :]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    train_dataset = TrainDataset(X_train, y_train)
    valid_dataset = TrainDataset(X_valid, y_valid)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

    for epoch in range(EPOCHS):
        train_loss, train_rmsle = train_fn(train_loader, model, criterion, optimizer, DEVICE)
        valid_loss, valid_rmsle, valid_preds = valid_fn(valid_loader, model, criterion, DEVICE)
        print(f'train_loss: {train_loss:.4f}, train_rmsle: {train_rmsle:.4f}')
        print(f'valid_loss: {valid_loss:.4f}, valid_rmsle: {valid_rmsle:.4f}')
    
    print('='*50)
    print(f'save model: models/nn_fold{i + 1}_.pth')
    print('='*50)
    torch.save(model.state_dict(), f"models/nn_fold{i + 1}_.pth")

    del model, criterion, optimizer
    gc.collect()

fold: 0


100%|██████████| 32/32 [00:00<00:00, 64.48it/s]


train_loss: 8.7285, train_rmsle: 8.7285
valid_loss: 7.7217, valid_rmsle: 7.7217


100%|██████████| 32/32 [00:00<00:00, 66.86it/s]


train_loss: 6.4025, train_rmsle: 6.4025
valid_loss: 4.9137, valid_rmsle: 4.9137


...

train_loss: 0.4602, train_rmsle: 0.4602
valid_loss: 0.5544, valid_rmsle: 0.5544


100%|██████████| 32/32 [00:00<00:00, 70.79it/s]


train_loss: 0.4529, train_rmsle: 0.4529
valid_loss: 0.5514, valid_rmsle: 0.5514


100%|██████████| 32/32 [00:00<00:00, 74.04it/s]


train_loss: 0.4497, train_rmsle: 0.4497
valid_loss: 0.5587, valid_rmsle: 0.5587
save model: models/nn_fold5_.pth


In [ ]:
list_preds_tmp = []
inference_dataset = TestDataset(X_inference)
inference_loader = DataLoader(inference_dataset, batch_size=BATCH_SIZE, shuffle=False)
for i in range(1, 6):
    model = Model(num_features, num_targets, hidden_size_1, hidden_size_2)
    model.load_state_dict(torch.load(f"models/nn_fold{i}_.pth"))
    model.to(DEVICE)
    predictions = inference_fn(model, inference_loader, DEVICE)
    list_preds_tmp.append(predictions)

In [ ]:
df_preds = pd.DataFrame({'model_1': np.squeeze(list_preds_tmp[0]),
                         'model_2': np.squeeze(list_preds_tmp[1]),
                         'model_3': np.squeeze(list_preds_tmp[2]),
                         'model_4': np.squeeze(list_preds_tmp[3]),
                         'model_5': np.squeeze(list_preds_tmp[4])})

df_preds['pred_avg'] = df_preds.mean(axis=1)

In [ ]:
df_preds.head()

,model_1,model_2,model_3,model_4,model_5,pred_avg
0,9.213768,9.007358,9.304264,9.436165,8.983576,9.189026
1,11.294427,11.266968,11.477156,10.573009,10.399199,11.002152
2,9.353668,9.358343,9.315969,9.422636,9.161180,9.322359
3,9.089067,9.378057,9.110122,8.970131,9.039038,9.117284
4,9.546414,9.110770,8.972805,9.207201,8.939746,9.155387


In [ ]:
sample_sub[COL_Y] = np.expm1(df_preds['pred_avg'])

In [ ]:
sample_sub.head()

,id,y
0,1,9788.110352
1,2,60002.156250
2,3,11184.337891
3,4,9110.419922
4,5,9464.291992


In [ ]:
sample_sub.to_csv('submit/submission_neural_network_fold_avg.csv', index=False)

In [ ]:
df_tmp = pd.read_csv('submit/submission_lightgbm_1-1-0.csv')
df_tmp.head()

,id,y
0,1,12936.384681
1,2,13570.382339
2,3,16072.926434
3,4,11621.427002
4,5,9691.119165


In [ ]:
df_ensemble = sample_sub.copy()
df_ensemble[COL_Y] = (sample_sub[COL_Y] + df_tmp[COL_Y]) / 2

In [ ]:
df_ensemble.head()

,id,y
0,1,12415.064899
1,2,18145.874763
2,3,13499.604330
3,4,11549.158813
4,5,10227.407239


In [ ]:
df_ensemble.to_csv('submit/submission_ensemble_nn_gbdt.csv', index=False)